In [0]:
from pyspark.sql import functions as F

# Raw Source Ingestion

In [0]:
ecom_oct_df = spark.read.csv("/Volumes/idc/idc_kaggle/ecom_data/2019-Oct.csv", header=True, inferSchema=True)


# Bronze layer process

In [0]:
ecom_oct_df = ecom_oct_df.withColumn("ingestion_time", F.current_timestamp())
ecom_oct_df.write.format("delta").mode("overwrite").save("/Volumes/idc/idc_kaggle/ecom_data/ecom_oct/bronze")

# Silver layer process

In [0]:
ecom_bronze_df = spark.read.format('delta').load('/Volumes/idc/idc_kaggle/ecom_data/ecom_oct/bronze')
display(ecom_bronze_df.limit(5))

In [0]:
ecom_silver_df = ecom_bronze_df.filter(F.col("price").between(0, 10000))\
    .dropDuplicates(["user_session","event_time"])\
    .withColumn("event_date", F.to_date(F.col("event_time")))\
    .withColumn("price_tier",
        F.when(F.col("price")<10,"budget")
        .when(F.col("price")<50,"affordable")
        .when(F.col("price")<100,"moderate")
        .when(F.col("price")<500,"expensive")
        .otherwise("luxury"))
    
ecom_silver_df.write.format("delta").mode("overwrite").save("/Volumes/idc/idc_kaggle/ecom_data/ecom_oct/silver")

# Golden data process

In [0]:
ecom_silver_df = spark.read.format('delta').load('/Volumes/idc/idc_kaggle/ecom_data/ecom_oct/silver')
display(ecom_silver_df.limit(5))

In [0]:
ecom_gold_prod = ecom_silver_df.groupBy("product_id")\
    .agg(
        F.countDistinct(F.when(F.col("event_type")=="view", F.col("user_id"))).alias("views"),
        F.countDistinct(F.when(F.col("event_type")=="purchase",F.col("user_id"))).alias("purchases"),
        F.sum(F.when(F.col("event_type")=="purchase",F.col("price"))).alias("revenue")
    ).withColumn("converstion_rate", F.col("purchases")/F.col("views")*100)

ecom_gold_prod.write.format("delta").mode("overwrite").save("/Volumes/idc/idc_kaggle/ecom_data/ecom_oct/gold/product")

In [0]:
ecom_gold_prod = spark.read.format('delta').load('/Volumes/idc/idc_kaggle/ecom_data/ecom_oct/gold/product')
display(ecom_gold_prod.limit(5))
